# Soduku Solver

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

2023-02-07 21:38:36.863471: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data Preprocessing

### Data Preprocessing - Part 1

In [3]:
data=pd.read_csv('sudoku.csv')[:5]
data

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


In [4]:
x=data['quizzes']
y=data['solutions']

print(x)
print()
print(y)

0    0043002090050090010700600430060020871900074000...
1    0401000501070039605200080000000000170009068008...
2    6001203840084590720000060050002640300700800069...
3    4972000001004000050000160986203000403009000000...
4    0059103080094030600275001000300002010008200070...
Name: quizzes, dtype: object

0    8643712593258497619712658434361925871986574322...
1    3461792581875239645296483719658324174729168358...
2    6951273841384596727248369158512647392739815469...
3    4972583161864397252537164986293815473759641828...
4    4659123781894735623275681497386452919548216372...
Name: solutions, dtype: object


In [5]:
X=[]
Y=[]
for i in x:
    x=np.array([int(j) for j in i]).reshape((9,9,1))
    X.append(x)

#centrer réduire
X=np.array(X)
X=X/9
X-=.5

for i in y:
        y = np.array([int(j) for j in i]).reshape((81,1)) - 1
        Y.append(y)

Y= np.array(Y)

print(X)
print(y)


[[[[-0.5       ]
   [-0.5       ]
   [-0.05555556]
   [-0.16666667]
   [-0.5       ]
   [-0.5       ]
   [-0.27777778]
   [-0.5       ]
   [ 0.5       ]]

  [[-0.5       ]
   [-0.5       ]
   [ 0.05555556]
   [-0.5       ]
   [-0.5       ]
   [ 0.5       ]
   [-0.5       ]
   [-0.5       ]
   [-0.38888889]]

  [[-0.5       ]
   [ 0.27777778]
   [-0.5       ]
   [-0.5       ]
   [ 0.16666667]
   [-0.5       ]
   [-0.5       ]
   [-0.05555556]
   [-0.16666667]]

  [[-0.5       ]
   [-0.5       ]
   [ 0.16666667]
   [-0.5       ]
   [-0.5       ]
   [-0.27777778]
   [-0.5       ]
   [ 0.38888889]
   [ 0.27777778]]

  [[-0.38888889]
   [ 0.5       ]
   [-0.5       ]
   [-0.5       ]
   [-0.5       ]
   [ 0.27777778]
   [-0.05555556]
   [-0.5       ]
   [-0.5       ]]

  [[-0.5       ]
   [ 0.05555556]
   [-0.5       ]
   [-0.5       ]
   [ 0.38888889]
   [-0.16666667]
   [-0.5       ]
   [-0.5       ]
   [-0.5       ]]

  [[ 0.16666667]
   [-0.5       ]
   [-0.5       ]
   [-0.5       ]
  

### Data Preprocessing - Part 2

In [6]:
def data_prepocessing():
    data=pd.read_csv('sudoku.csv')

    x=data['quizzes']
    y=data['solutions']
    X=[]
    Y=[]

    for i in x:
        x=np.array([int(j) for j in i]).reshape((9,9,1))
        X.append(x)
    X=np.array(X)
    X=X/9
    X-=.5

    for i in y:
            y = np.array([int(j) for j in i]).reshape((81,1)) - 1
            Y.append(y)
    Y= np.array(Y)

    del(x)
    del(y)

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    return x_train, x_test, y_train, y_test


### CNN

In [7]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape

def get_model():

    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(81*9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))

    return model

# Solving

In [8]:
import copy


In [9]:
#data prepocessing
x_train, x_test, y_train, y_test =data_prepocessing()

#fiting data
model = get_model()

adam = keras.optimizers.Adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

model.fit(x_train, y_train, batch_size=32, epochs=2)




KeyboardInterrupt



In [ ]:
def denorm(a):

    return (a+.5)*9

def norm(a):

    return (a/9)-.5

In [ ]:
def inference_sudoku(sample):

    '''
        This function solve the sudoku by filling blank positions one by one.
    '''

    feat = copy.copy(sample)

    while(1):

        out = model.predict(feat.reshape((1,9,9,1)))
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2)

        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)

        if(mask.sum()==0):
            break

        prob_new = prob*mask

        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)

    return pred

In [ ]:
def test_accuracy(feats, labels):

    correct = 0

    for i,feat in enumerate(feats):

        pred = inference_sudoku(feat)

        true = labels[i].reshape((9,9))+1

        if(abs(true - pred).sum()==0):
            correct += 1

    print(correct/feats.shape[0])

In [ ]:
test_accuracy(x_test[:100], y_test[:100])

In [ ]:
def solve_sudoku(game):

    game = game.replace('\n', '')
    game = game.replace(' ', '')
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    game = norm(game)
    game = inference_sudoku(game)
    return game

In [ ]:
game = '''
          0 8 0 0 3 2 0 0 1
          7 0 3 0 8 0 0 0 2
          5 0 0 0 0 7 0 3 0
          0 5 0 0 0 1 9 7 0
          6 0 0 7 0 9 0 0 8
          0 4 7 2 0 0 0 5 0
          0 2 0 6 0 0 0 0 9
          8 0 0 0 9 0 3 0 5
          3 0 0 8 2 0 0 1 0
      '''

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game)

In [ ]:
np.sum(game, axis=1)